<a href="https://colab.research.google.com/github/SatoruMorishita/automate1/blob/main/%E5%87%BA%E8%8D%B7%E8%A1%A8%E4%BD%9C%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **出荷表作成自動化**:



# 3か所への当日(8/1)出荷表を作成し、各場所にメールを自動送信する。

shipments.dbのテーブル名:shipments
未プラン.dbのテーブル名:orders

In [ ]:
#Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#shipments.dbからデータベースを抽出する

import sqlite3
import pandas as pd

conn = sqlite3.connect('/content/drive/MyDrive/Colab Notebooks/shipments.db')

#読み込んだデータをpandasのdataフレームにて表示
df = pd.read_sql_query("SELECT * FROM shipments", conn)

conn.close()

display(df.head())

,出荷ID,キャリア名,出荷日,名前,宛先,カートン数,重量,トラッキングナンバー,商品名,商品カテゴリ,着日
0,1,Carrier C,2025-08-01,ユーザー_1,〒1000001 東京都新宿区1丁目,9,11.44,TN1000000000,商品_1,Equipment,2025-08-06
1,2,Carrier C,2025-08-02,ユーザー_2,〒1000002 東京都新宿区2丁目,7,12.14,TN1000000001,商品_2,Apparel,2025-08-28
2,3,Carrier B,2025-08-02,ユーザー_3,〒1000003 東京都新宿区3丁目,4,14.19,TN1000000002,商品_3,Equipment,2025-08-12
3,4,Carrier A,2025-08-02,ユーザー_4,〒1000004 東京都新宿区4丁目,1,13.87,TN1000000003,商品_4,Apparel,2025-08-20
4,5,Carrier C,2025-08-01,ユーザー_5,〒1000005 東京都新宿区5丁目,2,5.20,TN1000000004,商品_5,Apparel,2025-08-31


In [ ]:
#Carrier A、Carrir B,Carrier Cの当日出荷表を作成していく
#キャリア名をフィルター、出荷日を当日

from datetime import date

conn = sqlite3.connect('/content/drive/MyDrive/Colab Notebooks/shipments.db')
df_updated = pd.read_sql_query("SELECT * FROM shipments", conn)

today = date.today().strftime('%Y-%m-%d')
#キャリアごとにフィルターして変数に格納(当日出荷で)
carrier_a_today = df_updated[(df_updated['キャリア名'] == 'Carrier A') & (df_updated['出荷日'] == '2025-08-01')]
carrier_b_today = df_updated[(df_updated['キャリア名'] == 'Carrier B') & (df_updated['出荷日'] == '2025-08-01')]
carrier_c_today = df_updated[(df_updated['キャリア名'] == 'Carrier C') & (df_updated['出荷日'] == '2025-08-01')]
display(carrier_a_today)

,出荷ID,キャリア名,出荷日,名前,宛先,カートン数,重量,トラッキングナンバー,商品名,商品カテゴリ,着日
8,9,Carrier A,2025-08-01,ユーザー_9,〒1000009 東京都新宿区9丁目,8,6.45,TN1000000008,商品_9,Apparel,2025-08-31
25,26,Carrier A,2025-08-01,ユーザー_26,〒1000026 東京都新宿区26丁目,1,7.78,TN1000000025,商品_26,Equipment,2025-08-21
27,28,Carrier A,2025-08-01,ユーザー_28,〒1000028 東京都新宿区28丁目,5,13.03,TN1000000027,商品_28,Footwear,2025-08-18
28,29,Carrier A,2025-08-01,ユーザー_29,〒1000029 東京都新宿区29丁目,9,13.67,TN1000000028,商品_29,Footwear,2025-08-20
44,45,Carrier A,2025-08-01,ユーザー_45,〒1000045 東京都新宿区45丁目,6,11.98,TN1000000044,商品_45,Apparel,2025-08-19


In [ ]:
#データベースから列を選んで順番を変更、キャリアごとに変数に格納
carrier_a_reordered = carrier_a_today[['キャリア名', '出荷日','着日', '名前', '宛先', 'カートン数','重量', 'トラッキングナンバー', '商品名', '商品カテゴリ']]
carrier_b_reordered = carrier_b_today[['キャリア名', '出荷日','着日', '名前', '宛先', 'カートン数','重量', 'トラッキングナンバー', '商品名', '商品カテゴリ']]
carrier_c_reordered = carrier_c_today[['キャリア名', '出荷日','着日', '名前', '宛先', 'カートン数','重量', 'トラッキングナンバー', '商品名', '商品カテゴリ']]

In [ ]:
from datetime import date
#出荷表名を変数に格納して、各キャリアごとにエクセルを作成
#シート名は出荷表
today = date.today().strftime('%Y%m%d')
file_namea = f"20250801_A社出荷表.xlsx"
file_nameb = f"20250801_B社出荷表.xlsx"
file_namec = f"20250801_C社出荷表.xlsx"
sheet_name = "出荷表"

carrier_a_reordered.to_excel(file_namea, sheet_name=sheet_name, index=False)
carrier_b_reordered.to_excel(file_nameb, sheet_name=sheet_name, index=False)
carrier_c_reordered.to_excel(file_namec, sheet_name=sheet_name, index=False)

print("Excel files created successfully.")

Excel files created successfully.


#列幅調整と格子、一行目ハイライトキャリアに重要な部分を黄色

In [ ]:
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side

'''#重複してるから不要
today = date.today().strftime('%Y%m%d')
file_namea = f"{today}_A社出荷表.xlsx"
file_nameb = f"{today}_B社出荷表.xlsx"
file_namec = f"{today}_C社出荷表.xlsx"
sheet_name = "出荷表"
'''

# 関数作成
def format_excel(file_name, sheet_name):
    #シート読み込む
    try:
        wb = load_workbook(file_name)
        ws = wb[sheet_name]

        #列幅調整
        for column in ws.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = (max_length + 15)
            ws.column_dimensions[column_letter].width = adjusted_width

        #ヘッダーに黄色ハイライトつける
        header_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid") # Yellow color
        for cell in ws[1]:
            cell.fill = header_fill

        # 格子をセルごとにつけていく
        thin_border = Border(left=Side(style='thin'),
                             right=Side(style='thin'),
                             top=Side(style='thin'),
                             bottom=Side(style='thin'))

        for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = thin_border


        wb.save(file_name)
        print(f"Formatted '{file_name}' successfully.")
    #File名が見つからない場合
    except FileNotFoundError:
        print(f"Error: File '{file_name}' not found.")
    #その他のエラー
    except Exception as e:
        print(f"An error occurred while formatting '{file_name}': {e}")

# 各キャリアごとの出荷表で関数を実行
format_excel(file_namea, sheet_name)
format_excel(file_nameb, sheet_name)
format_excel(file_namec, sheet_name)

Formatted '20250801_A社出荷表.xlsx' successfully.
Formatted '20250801_B社出荷表.xlsx' successfully.
Formatted '20250801_C社出荷表.xlsx' successfully.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 各社にファイルを添付しメールを送信

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.application import MIMEApplication
from email import encoders
import os
import pandas as pd
import sqlite3
from datetime import date

# メール設定
sender_email = "babolat.tennis911021@gmail.com" # 送信元メールアドレス
sender_password = "tgax rmxt jrig gqeb" # 送信元メールアドレスのアプリパスワード
receiver_email = "satoru.morishita915531@gmail.com" # 送信先メールアドレス


# メール送信関数
def send_email(file_name, sender_email, sender_password, receiver_email, body):
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = os.path.splitext(file_name)[0] # ファイル名から拡張子を除いたものを件名にする

    msg.attach(MIMEText(body, 'plain'))

    # ファイルを添付
    try:
        with open(file_name, 'rb') as f:
           attach = MIMEApplication(f.read(), _subtype='vnd.openxmlformats-officedocument.spreadsheetml.sheet')
           attach.add_header('Content-Disposition', 'attachment', filename=os.path.basename(file_name))
           msg.attach(attach)
    except FileNotFoundError:
        print(f"Error: Attachment file '{file_name}' not found.")
        return # ファイルが見つからない場合はメール送信をスキップ


    try:
        server = smtplib.SMTP('smtp.gmail.com', 587) # ご利用のメールサーバーに合わせて変更
        server.starttls()
        server.login(sender_email, sender_password)
        text = msg.as_string()
        server.sendmail(sender_email, receiver_email, text)
        server.quit()
        print(f"Email with attachment '{file_name}' sent successfully.")
    except Exception as e:
        print(f"Error sending email with attachment '{file_name}': {e}")


# shipments.dbからデータを抽出
conn = sqlite3.connect('/content/drive/MyDrive/Colab Notebooks/未プラン.db')
df_shipments = pd.read_sql_query("SELECT * FROM orders", conn)
conn.close()

# '出荷日' カラムをdatetime型に変換
df_shipments['出荷日'] = pd.to_datetime(df_shipments['出荷日'])


# 今日の日付をdatetime型で取得
today_dt = pd.to_datetime('2025-08-01')


# ファイル名のリスト
file_names = ["20250801_A社出荷表.xlsx", "20250801_B社出荷表.xlsx", "20250801_C社出荷表.xlsx"]

# 各ファイルに対してメールを送信
for file_name in file_names:
    # ファイル名からキャリア名を抽出
    carrier = file_name.split('_')[1].split('社')[0] # 'A社' -> 'A'
    carrier_name = f'Carrier {carrier}'


    # 未プラン.dbに当日出荷のデータが存在するか確認 (該当キャリアのみ)
    unplanned_shipments_exist = not df_shipments[(df_shipments['出荷日'] <= today_dt) & (df_shipments['キャリア名'] == carrier_name)].empty

    # メール本文　未プランありなしでパターン2つ
    if unplanned_shipments_exist:
       body = "お疲れ様です。\n\n本日出荷表ですが、未プランのものがあり、遅れます。\n\n現時点での出荷表を添付いたします。\n\nプラン完了しましたら即座に更新いたします。よろしくお願いします。\n\n"
    else:
       body = "お疲れ様です。\n\n本日の出荷表になります。ご確認よろしくお願いいたします。\n\n"

    send_email(file_name, sender_email, sender_password, receiver_email, body)

Email with attachment '20250801_A社出荷表.xlsx' sent successfully.
Email with attachment '20250801_B社出荷表.xlsx' sent successfully.
Email with attachment '20250801_C社出荷表.xlsx' sent successfully.


In [ ]:
######################################################################3